In [1]:
# Imports
import copy
import hashlib
import heapq
import re
import time
from collections import defaultdict
from collections import deque

start = time.time()

In [2]:
# Day 1
with open('input_01.txt') as f:
    data = f.read().splitlines()

instructions = data[0].split(', ')
directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
visited = {(0, 0)}
position = ((0, 1), (0, 0))
answer0102 = None

for instruction in instructions:
    turn = instruction[0]
    steps = int(instruction[1:])
    current_direction = position[0]
    
    new_direction = directions[(directions.index(current_direction) + (1 if turn == 'R' else -1)) % 4]
    for step in range(steps):
        position = (new_direction, (position[1][0] + new_direction[0], position[1][1] + new_direction[1]))
        if position[1] in visited and answer0102 is None:
            answer0102 = abs(position[1][0]) + abs(position[1][1])
        visited.add(position[1])

answer0101 = abs(position[1][0]) + abs(position[1][1])

time01 = time.time() - start

In [3]:
# Day 2
with open('input_02.txt') as f:
    data = f.read().splitlines()

# (r, c)
small_keypad = {
    (0, 0): '1',
    (0, 1): '2',
    (0, 2): '3',
    (1, 0): '4',
    (1, 1): '5',
    (1, 2): '6',
    (2, 0): '7',
    (2, 1): '8',
    (2, 2): '9' 
}

big_keypad = {
    (0, 2): '1',
    (1, 1): '2',
    (1, 2): '3',
    (1, 3): '4',
    (2, 0): '5',
    (2, 1): '6',
    (2, 2): '7',
    (2, 3): '8',
    (2, 4): '9',
    (3, 1): 'A',
    (3, 2): 'B',
    (3, 3): 'C',
    (4, 2): 'D'
}

moves = {
    'U': (-1, 0),
    'D': (1, 0),
    'L': (0, -1),
    'R': (0, 1)
}

answer0201 = ''
answer0202 = ''
small_position = (1, 1)
big_position = (2, 0)

for instruction in data:
    for move in instruction:
        new_small = (small_position[0] + moves[move][0], small_position[1] + moves[move][1])
        new_big = (big_position[0] + moves[move][0], big_position[1] + moves[move][1])
        small_position = new_small if new_small in small_keypad.keys() else small_position
        big_position = new_big if new_big in big_keypad.keys() else big_position

    answer0201 = answer0201 + small_keypad[small_position]
    answer0202 = answer0202 + big_keypad[big_position]

time02 = time.time() - start

In [4]:
# Day 3
with open('input_03.txt') as f:
    data = f.read().splitlines()

answer0301 = 0
answer0302 = 0

column_triangles = [[], [], []]

for line in data:
    A, B, C = [int(x) for x in line.split()]  
    if A + B > C and A + C > B and B + C > A:
        answer0301 += 1
    
    column_triangles[0].append(A)
    column_triangles[1].append(B)
    column_triangles[2].append(C)
    if len(column_triangles[0]) == 3:
        for i in range(3):
            A, B, C = column_triangles[i]
            if A + B > C and A + C > B and B + C > A:
                answer0302 += 1
        column_triangles = [[], [], []]

time03 = time.time() - start

In [5]:
# Day 4
with open('input_04.txt') as f:
    data = f.read().splitlines()

answer0401 = 0

def decrypt(encrypted_name):
    room_name = encrypted_name.split('[')[0]
    sector_id = int(re.findall(r'(\d+)', room_name)[0])
    room_name = room_name.replace('-' + str(sector_id), '')
    codes = []
    for char in room_name:
        codes.append(ord(char))
    rotations = sector_id % 26
    letters = []
    for code in codes:
        if code == 45:
            letters.append('-')
        else:
            if code + rotations <= 122:
                letters.append(chr(code + rotations))
            else:
                letters.append(chr(code + rotations - 26))
    decrypted_name = ''.join(letters)
    return decrypted_name

for room in data:
    letter_count = defaultdict(int)
    checksum_check = ''
    
    encrypted_name, checksum = room.split('[')
    checksum = checksum[:-1]
    sector_id = ''

    for letter in encrypted_name:
        if letter >= 'a' and letter <= 'z':
            letter_count[letter] += 1
        elif letter != '-':
            sector_id += letter
    sector_id = int(sector_id)
    
    while len(checksum_check) < 5:
        top_values = sorted([x for x in letter_count.values()])
        candidates = [k for k, v in letter_count.items() if v == top_values[-1]]
        next_letter = sorted(candidates, reverse = True)[-1]
        checksum_check = checksum_check + next_letter
        letter_count[next_letter] = 0

    if checksum == checksum_check:
        answer0401 += sector_id

    decrypted_room = decrypt(room)
    if decrypted_room.startswith('northpole'):
        answer0402 = sector_id

time04 = time.time() - start

In [6]:
# Day 5
with open('input_05.txt') as f:
    data = f.read().splitlines()

id = data[0]
answer0501 = []
answer0502 = [None, None, None, None, None, None, None, None]

i = 0
while len(answer0501) < 8 or None in answer0502:
    check = hashlib.md5((id + str(i)).encode()).hexdigest()
    if check[0:5] == '00000':
        answer0501.append(check[5:6])
        if check[5:6] in ['0', '1', '2', '3', '4', '5', '6', '7']:
            if answer0502[int(check[5:6])] == None:
                answer0502[int(check[5:6])] = check[6:7]
    i += 1

answer0501 = ''.join(answer0501)[0:8]
answer0502 = ''.join(answer0502)

time05 = time.time() - start

In [7]:
# Day 6
with open('input_06.txt') as f:
    data = f.read().splitlines()

answer0601 = ''
answer0602 = ''
for i in range(len(data[0])):
    letters = defaultdict(int)
    for line in data:
        letters[line[i]] += 1
    most_common_number = 0
    most_common_letter = ''
    least_common_number = len(data)
    least_common_letter = ''
    for k, v in letters.items():
        if v > most_common_number:
            most_common_number = v
            most_common_letter = k
        if v < least_common_number:
            least_common_number = v
            least_common_letter = k
    answer0601 = answer0601 + most_common_letter
    answer0602 = answer0602 + least_common_letter

time06 = time.time() - start

In [8]:
# Day 7
with open('input_07.txt') as f:
    data = f.read().splitlines()

answer0701 = 0
answer0702 = 0

for line in data:
    outside = True
    outsides = ['']
    insides = []
    abba_outside = False
    abba_inside = False

    for char in line:
        if char == '[':
            outside = False
            insides.append('')
            continue
        elif char == ']':
            outside = True
            outsides.append('')
            continue
        elif outside == True:
            outsides[-1] = outsides[-1] + char
        elif outside == False:
            insides[-1] = insides[-1] + char

    for list in [outsides, insides]:
        for string in list:
            for i in range(len(string) - 3):
                if string[i] != string[i + 1] and string[i] == string[i + 3] and string[i + 1] == string[i + 2]:
                    if list == outsides:
                        abba_outside = True
                    elif list == insides:
                        abba_inside = True
    
    bab = False
    babs = []
    for string in outsides:
        for i in range(len(string) - 2):
            if string[i] != string[i + 1] and string[i] == string[i + 2]:
                babs.append(string[i + 1] + string[i] + string[i + 1])
    for string in insides:
        for i in range(len(string) - 2):
            if string[i:i + 3] in babs:
                bab = True

    if abba_outside == True and abba_inside == False:
        answer0701 += 1
    if bab == True:
        answer0702 += 1

time07 = time.time() - start

In [9]:
# Day 8
with open('input_08.txt') as f:
    data = f.read().splitlines()

WIDE = 50
TALL = 6

screen = []
for row in range(TALL):
    rowvalues = [] 
    for column in range(WIDE):
        rowvalues.append(0)
    screen.append(rowvalues)

for line in data: 
    A, B = [int(x) for x in re.findall(r'(\d+)[a-zA-Z ]*(\d+)', line)[0]]
    if line.startswith('rect'):
        for b in range(B):
            for a in range(A):
                screen[b][a] = 1
    elif line.startswith('rotate column'):
        column_to_rotate = []
        for i in range(TALL):
            column_to_rotate.append(screen[i][A])
        for i in range(B):
            column_to_rotate = column_to_rotate[-1:] + column_to_rotate[:-1]
        for n, row in enumerate(screen):
            screen[n][A] = column_to_rotate[n]
    elif line.startswith('rotate row'):
        for i in range(B):
            screen[A] = screen[A][-1:] + screen[A][:-1]

answer0801 = 0
for row in screen:
    for light in row:
        if light == 1:
            answer0801 += 1

for line in screen:
    row = ''
    for char in line:
        if char == 1:
            row = row + 'X'
        else:
            row = row + ' '
    print(row)

answer0802 = 'CFLELOYFCS'

time08 = time.time() - start

 XX  XXXX X    XXXX X     XX  X   XXXXX  XX   XXX 
X  X X    X    X    X    X  X X   XX    X  X X    
X    XXX  X    XXX  X    X  X  X X XXX  X    X    
X    X    X    X    X    X  X   X  X    X     XX  
X  X X    X    X    X    X  X   X  X    X  X    X 
 XX  X    XXXX XXXX XXXX  XX    X  X     XX  XXX  


In [10]:
# Day 9
with open('input_09.txt') as f:
    data = f.read().splitlines()

answer0901 = 0
answer0902 = 0

def decompress(chars):
    decompressed = ''

    while chars:
        if chars[0] == '(':
            how_to_decompress = re.findall(r'\(\d*x\d*\)', chars)[0]
            lookahead, repetitions = how_to_decompress.split('x')
            lookahead = int(lookahead[1:])
            repetitions = int(repetitions[:-1])
            chars = chars[len(how_to_decompress):]
            string_to_decompress = chars[:lookahead]
            chars = chars[lookahead:]
            decompressed = decompressed + repetitions * string_to_decompress
        else:
            decompressed = decompressed + chars[0]
            chars = chars[1:]

    return decompressed

for line in data:
    answer0901 += len(decompress(line))

time09 = time.time() - start

In [11]:
# Day 10
with open('input_10.txt') as f:
    data = f.read().splitlines()

bots = dict()
outputs = dict()

for line in data:
    bots_found = re.findall(r'bot \d+', line)
    for bot in bots_found:
        bots[bot] = {'low': '', 'high': '', 'chips': []}
    outputs_founds = re.findall(r'output \d+', line)
    for output in outputs_founds:
        outputs[output] = []

for line in data:
    if line.startswith('value'):
        bots[re.findall(r'bot \d+', line)[0]]['chips'].append(int(re.findall(r'value (\d+)', line)[0]))
    else:
        bots[re.findall(r'(bot \d+) gives', line)[0]]['low'] = re.findall(r'low to (\bbot\b \d+|\boutput\b \d+)', line)[0]
        bots[re.findall(r'(bot \d+) gives', line)[0]]['high'] = re.findall(r'high to (\bbot\b \d+|\boutput\b \d+)', line)[0]

while True:
    if not [k for k, v in bots.items() if len(v['chips']) == 2]:
        break
    for k, v in bots.items():
        if len(v['chips']) == 2:
            if min(v['chips']) == 17 and max(v['chips']) == 61:
                answer1001 = k[4:]
            if v['high'].startswith('bot'):
                bots[v['high']]['chips'].append(max(v['chips']))
            elif v['high'].startswith('output'):
                outputs[v['high']].append(max(v['chips']))
            if v['low'].startswith('bot'):
                bots[v['low']]['chips'].append(min(v['chips']))
            elif v['low'].startswith('output'):
                outputs[v['low']].append(min(v['chips']))
            bots[k]['chips'] = []
            break

answer1002 = outputs['output 0'][0] * outputs['output 1'][0] * outputs['output 2'][0]

time10 = time.time() - start

In [ ]:
# Day 11
with open('input_11.txt') as f:
    data = f.read().splitlines()

#data = """The first floor contains a polonium generator, a thulium generator, a thulium-compatible microchip, a promethium generator, a ruthenium generator, a ruthenium-compatible microchip, a cobalt generator, a elerium generator, a elerium-compatible microchip, a dilithium generator, a dilithium-compatible microchip and a cobalt-compatible microchip.
#The second floor contains a polonium-compatible microchip and a promethium-compatible microchip.
#The third floor contains nothing relevant.
#The fourth floor contains nothing relevant.""".splitlines()

PATHS = []
heapq.heapify(PATHS)
floors = [0, 1]
for floor_number, contents in enumerate(data):
    if re.search(r'nothing', contents):
        floors.append(set())
    else:
        floor = set()
    for chip in re.findall(r'(\S+)-compatible microchip', contents):
        floor.add('chip-' + chip)
    for generator in re.findall(r'(\S+) generator', contents):
        floor.add('generator-' + generator)
    floors.append(floor)
heapq.heappush(PATHS, floors)

SEEN = set()

while PATHS:
    current_state = heapq.heappop(PATHS)
    current_cost, current_floor, floor1, floor2, floor3, floor4 = current_state
    current_contents = current_state[current_floor + 1]
    seen = (current_floor, tuple(floor1), tuple(floor2), tuple(floor3), tuple(floor4))

    if floor1 == floor2 == floor3 == set():
        answer1101 = current_cost
        break
    
    possible_floors = [f for f in [current_floor - 1, current_floor + 1] if f >= 1 and f <= 4]
    moves = []
    for floor in possible_floors:
        for content in current_contents:
            moves.append([floor, content, None])
        for combination in list(itertools.combinations(current_contents, 2)):
            moves.append([floor, combination[0], combination[1]])

    new_states = []
    for move in moves:
        new_floor = move[0]
        new_contents = copy.deepcopy(current_state[new_floor + 1])
        old_contents = copy.deepcopy(current_contents)
        if move[1] != None:
            new_contents.add(move[1])
            old_contents.remove(move[1])
        if move[2] != None:
            new_contents.add(move[2])
            old_contents.remove(move[2])
        new_state = []

        for i in range(len(current_state)):
            if i == 0:
                new_state.append(current_cost + 1)
            elif i == 1:
                new_state.append(new_floor)
            elif i == current_floor + 1:
                new_state.append(old_contents)
            elif i == new_floor + 1:
                new_state.append(new_contents)
            else:
                new_state.append(current_state[i])
        new_states.append(new_state)

    for state in new_states:
        state_safe = True
        for floor in state[2:]:
            generators_in_floor = [object[10:] for object in floor if object.startswith('generator-')]
            chips_in_floor = [object[5:] for object in floor if object.startswith('chip-')]
            if len(generators_in_floor) > 0:
                for chip in chips_in_floor:
                    if chip not in generators_in_floor:
                        state_safe = False
        if state_safe == True:
            seen = (state[1], tuple(state[2]), tuple(state[3]), tuple(state[4]), tuple(state[5]))
            if seen not in SEEN:
                SEEN.add(seen)
                heapq.heappush(PATHS, state)

time11 = time.time() - start

In [13]:
# Day 12
with open('input_12.txt') as f:
    data = f.read().splitlines()

def register_values(a, b, c, d):
    registers = {
        'a': a,
        'b': b,
        'c': c,
        'd': d
    }

    i = 0
    while True:
        if i + 1 > len(data):
            break
        line = data[i]
        if line.startswith('cpy'):
            value, register = re.findall(r'cpy (.+) (.+)', line)[0]
            if value in registers.keys():
                registers[register] = registers[value]
            else:
                registers[register] = int(value)
        elif line.startswith('inc'):
            register = re.findall(r'inc (.+)', line)[0]
            registers[register] += 1
        elif line.startswith('dec'):
            register = re.findall(r'dec (.+)', line)[0]
            registers[register] -= 1
        if line.startswith('jnz'):
            register, value = re.findall(r'jnz (.+) (.+)', line)[0]
            if (register in registers.keys() and registers[register] == 0) or register == '0':
                pass
            else:
                i = i + int(value)
                continue
        i += 1    
    
    return registers['a']

answer1201 = register_values(0, 0, 0, 0)
answer1202 = register_values(0, 0, 1, 0)

time12 = time.time() - start

In [14]:
# Day 13
with open('input_13.txt') as f:
    data = f.read().splitlines()

favorite_number = int(data[0])
start_position = (1, 1)
goal = (31, 39)

map = defaultdict(str)
costs = defaultdict(int)
costs[start_position] = 0

for x in range(max(goal) * 2):
    for y in range(max(goal) * 2):
        calculation = x*x + 3*x  + 2*x*y + y + y*y + favorite_number
        binary = bin(calculation)
        ones = len([x for x in binary if x == '1'])
        map[(x, y)] = '.' if ones % 2 == 0 else '#'

def possible_moves(position):
    current_cost = costs[position]
    next_cost = current_cost + 1
    
    possible_next_positions = [
        (position[0] + 1, position[1]), 
        (position[0] - 1, position[1]), 
        (position[0], position[1] + 1),
        (position[0], position[1] - 1)
    ]
    
    possible_moves = []
    for p in possible_next_positions:
        possible_moves.append((next_cost, p))
    
    return possible_moves

next_moves = []
heapq.heapify(next_moves)
for p in possible_moves(start_position):
    position = p[1]
    if position in map.keys() and map[position] != '#' and position not in costs.keys():
        heapq.heappush(next_moves, p)

while next_moves:
    if goal in costs.keys():
        break
    next_move = heapq.heappop(next_moves)
    next_cost = next_move[0]
    next_position = next_move[1]
    costs[next_position] = next_cost
    for p in possible_moves(next_position):
        position = p[1]
        cost = p[0]
        if position in map.keys() and map[position] != '#' and position not in costs.keys():
            heapq.heappush(next_moves, p)        

answer1301 = costs[goal]
answer1302 = len([k for k, v in costs.items() if v <= 50])

time13 = time.time() - start

In [ ]:
# Day 14
with open('input_14.txt') as f:
    data = f.read().splitlines()

salt = data[0]

def hash_n_times(input, iteration, number_of_iterations):
    hashvalue = hashlib.md5(input.encode()).hexdigest()
    if iteration == number_of_iterations:
        return hashvalue
    else:
        return hash_n_times(hashvalue, iteration + 1, number_of_iterations)

def get_keys(salt, iterations):
    keys = []

    hashes = defaultdict(str)

    i = 0
    while True:
        if len(keys) == 64:
            break
        
        if len(hashes) < i + 1001:
            for j in range(i, i + 10000):
                hashstring = salt + str(j)
                if hashstring not in hashes.keys():
                    hashes[hashstring] = hash_n_times(hashstring, 0, iterations)

        hashvalue = hashes[salt + str(i)]
        triplets = re.search(r'((.)\2{2,})', hashvalue)
        triplet = triplets[0][0:3] if triplets else None    
        key_found = False
        if triplet:
            for j in range(i + 1, i + 1001):
                if key_found == True:
                    break
                hashvalue = hashes[salt + str(j)]
                quintuplets = re.findall(r'((.)\2{4,})', hashvalue)
                for q in quintuplets:
                    if q[0][0] == triplet[0]:
                        keys.append(i)
                        key_found = True
                        break
        i += 1

    return keys

answer1401 = get_keys(salt, 0)[63]
answer1402 = get_keys(salt, 2016)[63]

time14 = time.time() - start

In [ ]:
# Day 15
with open('input_15.txt') as f:
    data = f.read().splitlines()

data2 = copy.copy(data)
data2.append('Disc #7 has 11 positions; at time=0, it is at position 0.')

def first_possible_time(input):

    discs = dict()

    for line in input:
        disc_no = int(re.findall(r'Disc #(\d+)', line)[0])
        positions = int(re.findall(r'has (\d+) positions', line)[0])
        start_position = int(re.findall(r'position (\d+).', line)[0])

        first = None
        for i in range(positions + 1):
            current = start_position
            for j in range(i):
                current = current + 1 if current < positions - 1 else 0
            time = 0
            while True:
                time += 1
                current = current + 1 if current < positions - 1 else 0
                if current == 0 and time == disc_no:
                    first = i
                    break
                if time > positions + 1:
                    break
            if first != None:
                break
        
    
        discs[disc_no] = (first, positions)

    while True:
        min_time = min([v[0] for k, v in discs.items()])
        max_time = max([v[0] for k, v in discs.items()])
        if min_time == max_time:
            return min_time
        for k, v in discs.items():
            if v[0] < max_time:
                discs[k] = (v[0] + v[1], v[1])
                break

answer1501 = first_possible_time(data)
answer1502 = first_possible_time(data2)

time15 = time.time() - start

In [ ]:
# Day 16
with open('input_16.txt') as f:
    data = f.read().splitlines()
data = data[0]

def transform_steps(input, desired_len):
    a = input
    b = input
    b = b[::-1]
    replaced_b = ''
    for char in b:
        replaced_b = replaced_b + ('0' if char == '1' else '1')
    result = a + '0' + replaced_b

    if len(result) < desired_len:
        return transform_steps(result, desired_len)
    else:
        return result[0:desired_len]

def create_checksum(input):
    pairs = [[]]
    for char in input:
        if len(pairs[-1]) == 2:
            pairs.append([])
        pairs[-1].append(char)
    checksum = ''
    for pair in pairs:
        checksum = checksum + ('1' if pair[0] == pair[1] else '0')
    if len(checksum) % 2 == 0:
        return create_checksum(checksum)
    else:
        return checksum

answer1601 = create_checksum(transform_steps(data, 272))
answer1602 = create_checksum(transform_steps(data, 35651584))

time16 = time.time() - start

In [ ]:
# Day 17
with open('input_17.txt') as f:
    data = f.read().splitlines()
code = data[0]

door_keys = 'bcdef'
door_dict = {
    0: 'U',
    1: 'D',
    2: 'L',
    3: 'R'
}
dir_dict = {
    'U': (0, -1),
    'D': (0, 1),
    'L': (-1, 0),
    'R': (1, 0)
}

start = (0, 0)
target = (3, 3)

PATHS = []
heapq.heapify(PATHS)
heapq.heappush(PATHS, (0, start, code))

answer1701_found = False
answer1702 = 0

while PATHS:
    current_path = heapq.heappop(PATHS)
    current_cost = current_path[0]
    current_pos = current_path[1]
    current_charpath = current_path[2]

    if current_pos == target:
        if answer1701_found == False:
            answer1701 = current_charpath.replace(code, '')
            answer1701_found = True
        answer1702 = max(current_cost, answer1702)
        continue

    hashvalue = hashlib.md5(current_charpath.encode()).hexdigest()
    open_doors = []
    for d, key in enumerate(hashvalue[0:4]):
        if key in door_keys:
            open_doors.append(door_dict[d])

    for door in open_doors:
        new_pos = (current_pos[0] + dir_dict[door][0], current_pos[1] + dir_dict[door][1])
        if min(new_pos) < 0 or max(new_pos) > 3:
            pass
        else:
            heapq.heappush(PATHS, (current_cost + 1, new_pos, current_charpath + door))

time17 = time.time() - start

In [ ]:
# Day 18
with open('input_18.txt') as f:
    data = f.read().splitlines()

first_row = data[0]
number_of_rows = 400000

grid = []
row = []
for char in first_row:
     row.append(char)
grid.append(row)

for i in range(number_of_rows - 1):
    row = []
    for j in range(len(grid[0])):
        row.append('X')
    grid.append(row)

while True:  
    print()
    print("Length of grid:", len(grid))
    empty_rows = 0
    keep_going = False
    for row in grid:
        if 'X' in row:
            empty_rows += 1
            keep_going = True
    if keep_going == False:
        break
    print("Rows with unknowns:", empty_rows)

    for r, row in enumerate(grid):
        for c, col in enumerate(row):
            if col == 'X':
                if c - 1 < 0:
                    left = '.'
                else:
                    left = grid[r - 1][c - 1]
                if c + 1 > len(grid[0]) - 1:
                    right = '.'
                else:
                    right = grid[r - 1][c + 1]
                center = grid[r - 1][c]
                if (left, center, right) in [('^', '^', '.'), ('.', '^', '^'), ('^', '.', '.'), ('.', '.', '^')]:
                    grid[r][c] = '^'
                elif 'X' not in (left, center, right):
                    grid[r][c] = '.'
                else:
                    pass    

answer1801 = 0
for row in grid:
    for char in row:
        if char == '.':
            answer1801 += 1

print(answer1801)

answer1802 = 0
for row in grid:
    for char in row:
        if char == '.':
            answer1802 += 1

for i in range(50):
    print(grid[i])

time18 = time.time() - start


Length of grid: 40000
Rows with unknowns: 39999

Length of grid: 40000
2001938
['^', '^', '^', '^', '.', '.', '.', '.', '.', '.', '^', '.', '.', '.', '^', '.', '.', '^', '.', '.', '.', '.', '^', '^', '^', '.', '^', '^', '^', '.', '^', '.', '^', '^', '^', '^', '^', '^', '.', '.', '^', '.', '.', '.', '^', '^', '.', '.', '.', '^', '^', '^', '.', '^', '^', '.', '.', '.', '.', '^', '.', '.', '^', '^', '^', '.', '^', '.', '^', '^', '.', '.', '.', '^', '.', '^', '.', '.', '.', '^', '^', '.', '^', '^', '^', '.', '^', '^', '^', '^', '.', '^', '^', '.', '^', '.', '.', '^', '.', '^']
['^', '.', '.', '^', '^', '.', '.', '.', '.', '^', '.', '^', '.', '^', '.', '^', '^', '.', '^', '.', '.', '^', '^', '.', '^', '.', '^', '.', '^', '.', '.', '.', '^', '.', '.', '.', '.', '^', '^', '^', '.', '^', '.', '^', '^', '^', '^', '.', '^', '^', '.', '^', '.', '^', '^', '^', '.', '.', '^', '.', '^', '^', '^', '.', '^', '.', '.', '.', '^', '^', '^', '.', '^', '.', '.', '.', '^', '.', '^', '^', '^', '.', '^', '.'

In [ ]:
# Day 19
with open('input_19.txt') as f:
    data = f.read().splitlines()

from collections import deque

number_of_elves = int(data[0])

elves = deque([])
for elf in range(number_of_elves):
    elves.append(elf + 1)
while len(elves) != 1:
    taking = elves.popleft()
    giving = elves.popleft()
    elves.append(taking)
answer1901 = elves[0]

first_half = deque([])
second_half = deque([])
for elf in range(number_of_elves):
    second_half.append(elf + 1)

while True:
    if len(second_half) == 1:
        break
    while len(second_half) - len(first_half) >= 2:
        first_half.append(second_half.popleft())
    second_half.popleft()
    second_half.append(first_half.popleft())
answer1902 = second_half[0]

time19 = time.time() - start

1834471
1420064


In [ ]:
# Day 20
with open('input_20.txt') as f:
    data = f.read().splitlines()

min_ip = 0
max_ip = 4294967295

blacklists = []
whitelists = []
position = 0

for line in data:
    low, high = [int(x) for x in re.findall(r'(\d+)-(\d+)', line)[0]]
    blacklists.append((low, high))
blacklists.sort()

for list in blacklists:
    low, high = list
    if position < low:
        whitelists.append((position, low - 1))
    position = max(high + 1, position)
if position < max_ip:
    whitelists.append((position, max_ip))

answer2001 = max_ip
answer2002 = 0

for list in whitelists:
    answer2001 = min(list[0], answer2001)
    answer2002 += (list[1] - list[0] + 1)

time20 = time.time() - start

In [ ]:
# Day 21
with open('input_21.txt') as f:
    data = f.read().splitlines()

to_scramble = 'abcdefgh'

def scrambling_function(text, instruction):
    new_text = [x for x in text]
    if instruction.startswith('swap position'):
        X, Y = [int(x) for x in re.findall(r'position (\d+)', instruction)]
        new_text[X] = text[Y]
        new_text[Y] = text[X]
    elif instruction.startswith('swap letter'):
        first, second = re.findall(r'letter (\S+)', instruction)
        for n, char in enumerate(new_text):
            if char == first:
                new_text[n] = second
            elif char == second:
                new_text[n] = first
    elif instruction.startswith('rotate left'):
        steps = int(re.findall(r'(\d+) step', instruction)[0])
        for s in range(steps):
            new_text = new_text[1:] + [new_text[0]]
    elif instruction.startswith('rotate right'):
        steps = int(re.findall(r'(\d+) step', instruction)[0])
        for s in range(steps):
            new_text = [new_text[-1]] + new_text[:-1]
    elif instruction.startswith('rotate based'):
        letter = re.findall(r'letter (\S+)', instruction)[0]
        index = new_text.index(letter)
        for s in range(index + 1 + (1 if index >= 4 else 0)):
            new_text = [new_text[-1]] + new_text[:-1]
    elif instruction.startswith('reverse'):
        X, Y = [int(x) for x in re.findall(r'(\S+) through (\S+)', instruction)[0]]
        reversed_list = []
        for n in range(Y - X + 1):
            reversed_list.append(new_text[Y - n])
        new_text = new_text[0:X] + reversed_list + new_text[Y + 1:]
    elif instruction.startswith('move'):
        X, Y = [int(x) for x in re.findall(r'(\d+) to position (\d+)', instruction)[0]]
        removed = new_text[X]
        new_text = new_text[0:X] + new_text[X+1:]
        new_text.insert(Y, removed)
    new_text = ''.join(new_text)
    return new_text

for line in data:
    to_scramble = scrambling_function(to_scramble, line)

answer2101 = to_scramble
answer2102 = 0

time21 = time.time() - start

In [ ]:
# Day 22
with open('input_22.txt') as f:
    data = f.read().splitlines()

nodes = defaultdict(dict)
pairs = []

for line in data:
    if not re.search(r'node-x', line):
        continue
    x, y, size, used, avail = [int(x) for x in re.findall(r'node-x(\d+)-y(\d+)\s+(\d+)T\s+(\d+)T\s+(\d+)T', line)[0]]
    nodes[(x, y)] = {'size': size, 'used': used, 'avail': avail}

for kA, vA in nodes.items():
    for kB, vB in nodes.items():
        if kA != kB and vA['used'] > 0 and vA['used'] <= vB['avail']:
            pairs.append((kA, kB))

answer2201 = len(pairs)
answer2202 = 0

time22 = time.time() - start

In [ ]:
# Day 23
with open('input_23.txt') as f:
    data2 = f.read().splitlines()

def register_values(data, a, b, c, d, plus_or_multiply):
    registers = {
        'a': a,
        'b': b,
        'c': c,
        'd': d
    }

    i = 0
    while True:
        if i + 1 > len(data):
            break        
        line = data[i]
        if line.startswith('cpy'):
            value, register = re.findall(r'cpy (.+) (.+)', line)[0]
            if register not in registers.keys():
                pass
            elif value in registers.keys():
                registers[register] = registers[value]
            else:
                registers[register] = int(value)
        elif line.startswith('inc'):
            register = re.findall(r'inc (.+)', line)[0]
            if plus_or_multiply == 'plus':
                registers[register] += 1
            elif plus_or_multiply == 'multiply':
                registers[register] *= 2
        elif line.startswith('dec'):
            register = re.findall(r'dec (.+)', line)[0]
            registers[register] -= 1
        elif line.startswith('jnz'):
            register, value = re.findall(r'jnz (.+) (.+)', line)[0]
            if (register in registers.keys() and registers[register] == 0) or register == '0':
                pass
            else:
                if value in registers.keys():
                    value = registers[value]
                else:
                    value = int(value)
                i = i + value
                continue
        elif line.startswith('tgl'):
            to_pass = False
            value = re.findall(r'tgl (.+)', line)[0]
            if value in registers.keys():
                value = i + registers[value]
            else:
                value = i + int(value)
            if value < 0 or value >= len(data):
                to_pass = True
            else:
                data_value = data[value]
            if to_pass == True:
                pass
            elif data_value.startswith('inc'):
                data[value] = data[value].replace('inc', 'dec')
            elif data_value.startswith('dec') or data_value.startswith('tgl'):
                data[value] = data[value].replace('dec', 'inc')
                data[value] = data[value].replace('tgl', 'inc')
            elif data_value.startswith('jnz'):
                data[value] = data[value].replace('jnz', 'cpy')
            elif data_value.startswith('cpy'):
                data[value] = data[value].replace('cpy', 'jnz')            
        i += 1    
    
    return registers['a']

with open('input_23.txt') as f:
    data2 = f.read().splitlines()
answer2301 = register_values(data2, 7, 0, 0, 0, 'plus')

print(answer2301)

with open('input_23.txt') as f:
    data2 = f.read().splitlines()
answer2302 = register_values(data2, 12, 0, 0, 0, 'multiply')

print(answer2302)

time23 = time.time() - start

11500


KeyboardInterrupt: 

In [ ]:
# Day 24
with open('input_24.txt') as f:
    data = f.read().splitlines()

answer2401 = 0
answer2402 = 0

time24 = time.time() - start

In [ ]:
# Day 25
with open('input_25.txt') as f:
    data = f.read().splitlines()

answer2501 = 0
answer2502 = 0

time25 = time.time() - start

In [ ]:
# Answers
def pretty_answer(day, part):
    day_text = ('00' + str(day))[-2:]
    part_text = ('00' + str(part))[-2:]
    answer_text = str(globals()['answer' + day_text + part_text])
    output_text = 'Part ' + part_text + ': ' + answer_text
    print(output_text)

def timed_answer(day):
    day_text = ('00' + str(day))[-2:]
    print('DAY', day_text)
    for p in range(1, 3):
        pretty_answer(day, p)
    print('Elapsed time:', round(globals()['time' + day_text] / 60, 3), 'minutes')
    print()

for d in range(1, 26):
    timed_answer(d)

DAY 01
Part 01: 243
Part 02: 142
Elapsed time: 0.001 minutes

DAY 02
Part 01: 74921
Part 02: A6B35
Elapsed time: 0.002 minutes

DAY 03
Part 01: 869
Part 02: 1544
Elapsed time: 0.002 minutes

DAY 04
Part 01: 361724
Part 02: 482
Elapsed time: 0.008 minutes

DAY 05
Part 01: f97c354d
Part 02: 863dde27
Elapsed time: 2.096 minutes

DAY 06
Part 01: xhnqpqql
Part 02: brhailro
Elapsed time: 2.098 minutes

DAY 07
Part 01: 115
Part 02: 231
Elapsed time: 2.104 minutes

DAY 08
Part 01: 0
Part 02: 0
Elapsed time: 2.105 minutes

DAY 09
Part 01: 120765
Part 02: 0
Elapsed time: 2.106 minutes

DAY 10
Part 01: 147
Part 02: 55637
Elapsed time: 2.107 minutes

DAY 11
Part 01: 0
Part 02: 0
Elapsed time: 2.108 minutes

DAY 12
Part 01: 318117
Part 02: 9227771
Elapsed time: 3.836 minutes

DAY 13
Part 01: 92
Part 02: 124
Elapsed time: 3.841 minutes

DAY 14
Part 01: 35186
Part 02: 22429
Elapsed time: 7.491 minutes

DAY 15
Part 01: 148737
Part 02: 2353212
Elapsed time: 7.653 minutes

DAY 16
Part 01: 10010100110011